<a href="https://colab.research.google.com/github/priyanshgupta1998/All_codes/blob/master/intursion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Signature based intrusion detection system

In [0]:

## Preliminaries
import numpy as np 
import matplotlib.pyplot as plt

import pandas as pd
import urllib
import csv


In [0]:
from sklearn import datasets, preprocessing, feature_extraction

In [0]:

from sklearn import linear_model, svm, metrics, ensemble, tree
from sklearn.decomposition import PCA

In [0]:
from sklearn import cross_validation

In [0]:
# Helper functions
def folds_to_split(data,targets,train,test):
    data_tr = pd.DataFrame(data).iloc[train]
    data_te = pd.DataFrame(data).iloc[test]
    labels_tr = pd.DataFrame(targets).iloc[train]
    labels_te = pd.DataFrame(targets).iloc[test]
    return [data_tr, data_te, labels_tr, labels_te]

In [0]:

#let's load the data
train_data = pd.read_csv('/home/kddcup.data_10_percent_corrected.csv')
test_data =  pd.read_csv('/home/corrected.csv')

In [0]:
#Place both dataset into a dataframe
train_multiclass = pd.read_csv('/home/kddcup.data_10_percent_corrected.csv', quotechar=',', skipinitialspace=True , names=['Duration', 'protocol_type', 'Service', 'Flag', 'src_bytes', 'dst_bytes', 'Land', 'wrong_fragment', 'Urgent', 'Hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'Count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','Class'])

In [57]:
train_multiclass.iloc[:,:-1].head(4)  # leave the last column of the dataset

,Duration,protocol_type,Service,Flag,src_bytes,dst_bytes,Land,wrong_fragment,Urgent,Hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0


In [14]:
train_multiclass.shape

(494021, 42)

In [15]:
train_multiclass.head()

,Duration,protocol_type,Service,Flag,src_bytes,dst_bytes,Land,wrong_fragment,Urgent,Hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [16]:
train_multiclass.tail()

,Duration,protocol_type,Service,Flag,src_bytes,dst_bytes,Land,wrong_fragment,Urgent,Hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.
494020,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.


In [0]:
test_multiclass = pd.read_csv('/home/corrected.csv', quotechar=',', skipinitialspace=True, names=['Duration', 'protocol_type', 'Service', 'Flag', 'src_bytes', 'dst_bytes', 'Land', 'wrong_fragment', 'Urgent', 'Hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'Count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','Class'])

In [20]:
test_multiclass.shape

(311029, 42)

In [26]:
test_multiclass.head()

,Duration,protocol_type,Service,Flag,src_bytes,dst_bytes,Land,wrong_fragment,Urgent,Hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
0,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
3,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
4,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.


In [27]:
test_multiclass.tail()

,Duration,protocol_type,Service,Flag,src_bytes,dst_bytes,Land,wrong_fragment,Urgent,Hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
311024,0,udp,private,SF,105,147,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,normal.
311025,0,udp,private,SF,105,147,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,normal.
311026,0,udp,private,SF,105,147,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,normal.
311027,0,udp,private,SF,105,147,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,normal.
311028,0,udp,private,SF,105,147,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,normal.


#Dos attack**

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='smurf.')|(train_multiclass['Class'] =='neptune.') | (train_multiclass['Class'] =='back.') | (train_multiclass['Class'] =='teardrop.') |(train_multiclass['Class'] =='pod.')| (train_multiclass['Class']=='land.'),'Class'] = 'DOS_ATTACK'

#Probe attack

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='satan.')|(train_multiclass['Class'] =='ipsweep.') | (train_multiclass['Class'] =='portsweep.') | (train_multiclass['Class'] =='nmap.'),'Class'] = 'probe'

#R2L Attack

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='spy.')|(train_multiclass['Class'] =='phf.')|(train_multiclass['Class'] =='multihop.')|(train_multiclass['Class'] =='ftp_write.') | (train_multiclass['Class'] =='imap.') | (train_multiclass['Class'] =='warezmaster.') |(train_multiclass['Class'] =='guess_passwd.')| (train_multiclass['Class']=='warezclient.'),'Class'] = 'r2l'

#U2R attack

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='buffer_overflow.')|(train_multiclass['Class'] =='rootkit.') | (train_multiclass['Class'] =='loadmodule.') | (train_multiclass['Class'] =='perl.'),'Class']='u2r'

#Normal 

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='normal.'),'Class'] = 'normal'

`Now make a class for similar kind of attacks  IN test_set`

#For test data

#Dos attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='smurf.')|(test_multiclass['Class'] =='neptune.') | 
                    (test_multiclass['Class'] =='back.') | (test_multiclass['Class'] =='teardrop.') |
                    (test_multiclass['Class'] =='pod.')| (test_multiclass['Class']=='land.')|
                   (test_multiclass['Class']=='apache2.')|(test_multiclass['Class']=='udpstorm.')|
                   (test_multiclass['Class']=='processtable.')|(test_multiclass['Class']=='mailbomb.'),'Class'] ='DOS_ATTACK'


#r2l attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='guess_passwd.')|(test_multiclass['Class'] =='ftp_write.')|
                    (test_multiclass['Class'] =='imap.')|(test_multiclass['Class'] =='phf.') | 
                    (test_multiclass['Class'] =='multihop.') | 
                    (test_multiclass['Class'] =='warezmaster.') |(test_multiclass['Class'] =='snmpgetattack.')| 
                    (test_multiclass['Class']=='named.')|(test_multiclass['Class'] =='xlock.')|
                    (test_multiclass['Class'] =='xsnoop.')|(test_multiclass['Class'] =='sendmail.')|
                    (test_multiclass['Class'] =='httptunnel.')|(test_multiclass['Class'] =='worm.')|
                    (test_multiclass['Class'] =='snmpguess.'),'Class'] = 'r2l'

#probe attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='satan.')|(test_multiclass['Class'] =='ipsweep.') | (test_multiclass['Class'] =='portsweep.') | (test_multiclass['Class'] =='nmap.')|
                    (test_multiclass['Class'] =='saint.')|(test_multiclass['Class'] =='mscan.'),'Class'] = 'probe'


#u2r attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='buffer_overflow.')|(test_multiclass['Class'] =='rootkit.') | 
                    (test_multiclass['Class'] =='loadmodule.') | (test_multiclass['Class'] =='xterm.')|
                    (test_multiclass['Class'] =='sqlattack.')|(test_multiclass['Class'] =='ps.')|
                    (test_multiclass['Class'] =='perl.'),'Class']='u2r'

#normal attacks

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='normal.'),'Class'] = 'normal'

#Encoding

In [0]:
attr_encoder = feature_extraction.DictVectorizer(sparse=False)

In [91]:
attr_encoder

DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [0]:
label_encoder = preprocessing.LabelEncoder()



```
Train the model upto second last column of the dataset.
Because we will use last column/element for model prediction.
```



In [95]:
label_encoder

LabelEncoder()

In [97]:
type(label_encoder)

sklearn.preprocessing.label.LabelEncoder

In [0]:
train_data_df_m = attr_encoder.fit_transform(train_multiclass.iloc[:,:-1].T.to_dict().values())

In [100]:
type(train_data_df_m)

numpy.ndarray

In [101]:
train_data_df_m[0:2]

array([[8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 5.45e+03,
        9.00e+00, 0.00e+00, 0.00e+00, 1.10e-01, 

In [102]:
len(train_data_df_m)

494021

In [103]:
np.shape(train_data_df_m)  # 494021 rows and each row has 118 elements

(494021, 118)

In [0]:
train_target_df_m= label_encoder.fit_transform(train_multiclass.iloc[:,-1])  #prediction purose __>target_data

In [105]:
len(train_target_df_m)

494021

In [106]:
np.shape(train_target_df_m)  #494021 columns and 1 row

(494021,)

In [107]:
print(train_target_df_m)

[1 1 1 ... 1 1 1]


#Decoding

#Convert into their respective Dataframe by uusing pandas library

In [108]:
train_data_decoded_m = pd.DataFrame(train_data_df_m)
train_data_decoded_m.head(5)

,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,181.0,8.0,0.0,0.0,0.0,0.0,0.0
1,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,239.0,8.0,0.0,0.0,0.0,0.0,0.0
2,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,235.0,8.0,0.0,0.0,0.0,0.0,0.0
3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,219.0,6.0,0.0,0.0,0.0,0.0,0.0
4,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,217.0,6.0,0.0,0.0,0.0,0.0,0.0


In [109]:
len(train_data_decoded_m)

494021

In [110]:
train_target_decoded_m = pd.DataFrame(train_target_df_m)
train_target_decoded_m.head(5)

,0
0,1
1,1
2,1
3,1
4,1


In [111]:
len(train_target_decoded_m)

494021

In [112]:
np.shape(train_target_decoded_m)

(494021, 1)

#Encoding of test data

In [0]:
test_data_df_m = attr_encoder.transform(test_multiclass.iloc[:,:-1].T.to_dict().values())


In [116]:
test_data_df_m[0:2]  #2 rows/Bloks with complete items

array([[1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.46e+02,
        2.55e+02, 1.00e-02, 0.00e+00, 0.00e+00, 

In [117]:
len(test_data_df_m)

311029

In [118]:
np.shape(test_data_df_m)

(311029, 118)

In [0]:
test_target_df_m = label_encoder.fit_transform(test_multiclass.iloc[:,-1])  #takes the target from the test dataset

In [120]:
len(test_target_df_m )

311029

In [121]:
np.shape(test_target_df_m )

(311029,)

In [0]:
test_data_decoded_m = pd.DataFrame(test_data_df_m)


In [123]:
test_data_decoded_m.head(4)

,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,105.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,105.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,105.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,105.0,2.0,0.0,0.0,0.0,0.0,0.0


In [0]:
test_target_decoded_m = pd.DataFrame(test_target_df_m)

In [126]:
test_target_decoded_m.head()

,0
0,1
1,1
2,1
3,3
4,3


In [128]:
np.shape(test_target_decoded_m)

(311029, 1)

In [130]:
print(np.shape(train_data_decoded_m))
print(np.shape(test_data_decoded_m))

(494021, 118)
(311029, 118)


#Perfroming Feature Reduction using PCA

In [0]:
#load some modules to help
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

In [0]:
train_data_pca2 = PCA(n_components=29).fit_transform(train_data_decoded_m)

In [133]:
len(train_data_pca2)

494021

In [135]:
np.shape(train_data_pca2)  #Now we have only 29 featres/columns in the train dataset

(494021, 29)

In [0]:
test_data_pca2 = PCA(n_components=29).fit_transform(test_data_decoded_m)

In [137]:
len(test_data_pca2)

311029

In [138]:
np.shape(test_data_pca2)   #Now we have only 29 featres/columns in the test dataset

(311029, 29)

In [140]:
train_data_pca_df2 = pd.DataFrame(train_data_pca2)
train_data_pca_df2.head(5)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-2844.610205,4581.635636,-21.689210,-474.329333,-38.352353,186.033679,-88.184465,-0.121237,-0.156435,-0.172176,...,-0.225079,0.003933,-0.493636,-0.007430,0.288167,0.047651,-0.024740,0.060569,-0.025038,0.016440
1,-2786.609945,-382.365152,-21.221032,-472.960277,-42.506158,173.646882,-83.783791,-0.106133,-0.151558,-0.166487,...,-0.235544,0.005564,-0.469927,0.019616,0.276087,0.058783,-0.019201,0.057871,-0.038222,0.020209
2,-2790.609995,468.633641,-21.539673,-469.624595,-46.106684,161.082040,-79.313554,-0.101148,-0.147239,-0.162089,...,-0.232838,0.005879,-0.450467,0.026446,0.264932,0.060443,-0.016394,0.054965,-0.040956,0.020961
3,-2806.609997,468.633472,-21.591309,-469.313053,-50.278016,148.052794,-74.759959,-0.094547,-0.143475,-0.159052,...,-0.224318,0.005512,-0.432843,0.022419,0.254208,0.057135,-0.017724,0.050542,-0.039231,0.021209
4,-2808.610038,1163.632276,-21.888843,-466.030115,-53.893393,135.492738,-70.291619,-0.089306,-0.139218,-0.154979,...,-0.218139,0.005477,-0.414449,0.024237,0.243337,0.056433,-0.015632,0.047619,-0.039302,0.021285


In [143]:
test_data_pca_df2 = pd.DataFrame(test_data_pca2)
test_data_pca_df2.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-1628.211606,-597.542092,16.470322,-341.322684,-102.610636,61.563490,5.814997,0.054780,0.026298,-0.531678,...,-0.001606,-0.042323,0.019440,-0.037529,-0.019196,0.042395,-0.003799,-0.003363,0.009414,-0.010332
1,-1628.211606,-597.542092,16.470322,-341.322684,-102.610636,61.563490,5.814997,0.054780,0.026298,-0.531678,...,-0.001606,-0.042323,0.019440,-0.037529,-0.019196,0.042395,-0.003799,-0.003363,0.009414,-0.010332
2,-1628.211606,-597.542092,16.470322,-341.322684,-102.610636,61.563490,5.814997,0.054780,0.026298,-0.531678,...,-0.001606,-0.042323,0.019440,-0.037529,-0.019196,0.042395,-0.003799,-0.003363,0.009414,-0.010332
3,-1628.211636,-597.543501,16.332525,-339.934119,-102.420856,61.434743,5.797819,0.054665,0.026183,-0.529278,...,-0.001631,-0.042350,0.019437,-0.037534,-0.019203,0.042416,-0.003825,-0.003357,0.009438,-0.010308
4,-1628.211636,-597.543501,16.332523,-339.934107,-102.420867,61.434744,5.797805,0.054666,0.026315,-0.532738,...,-0.001133,-0.043129,0.020566,-0.031604,-0.018794,0.037750,-0.004073,-0.003061,0.011198,-0.009899


In [144]:
print( train_data_pca_df2.shape)
print(test_data_pca_df2.shape)

(494021, 29)
(311029, 29)


#Normalizing the Data Sets

In [0]:

#Creating our scaler and applyting it to our dataset after feature reduction
standard_scaler = preprocessing.StandardScaler()

In [0]:
train_ratio_standard_scaled_values2 = standard_scaler.fit_transform(train_data_pca_df2.values)

In [147]:
len(train_ratio_standard_scaled_values2 )

494021

In [148]:
np.shape(train_ratio_standard_scaled_values2)

(494021, 29)

Bring the dataset feature within a range 

In [152]:
train_ratio_standard_scaled_values2[0:2]

array([[-2.87852761e-03,  1.38669485e-01, -3.06061171e-02,
        -1.44629472e+00, -3.93015682e-01,  4.00770128e+00,
        -3.01834671e+00, -4.50730489e-02, -2.00194199e-01,
        -2.72087937e-01, -2.04971316e+00, -3.10567712e+00,
        -9.82462079e-01,  4.03766446e+00,  1.66522959e+00,
        -2.69310501e+00,  1.70247953e+00,  3.21572066e+00,
        -2.70574848e+00, -2.30407659e+00,  4.09814369e-02,
        -5.49720772e+00, -9.96441839e-02,  4.25638944e+00,
         7.17135823e-01, -4.47806693e-01,  1.12383100e+00,
        -5.41858325e-01,  3.87295917e-01],
       [-2.81983579e-03, -1.15728056e-02, -2.99454606e-02,
        -1.44212028e+00, -4.35581791e-01,  3.74085400e+00,
        -2.86772198e+00, -3.94578014e-02, -1.93953277e-01,
        -2.63097938e-01, -1.95472987e+00, -3.07458898e+00,
        -8.72841362e-01,  3.91025825e+00,  1.62814290e+00,
        -2.79085446e+00,  1.67992154e+00,  3.14109068e+00,
        -2.58178600e+00, -2.41120321e+00,  5.79774152e-02,
        -5.23

In [153]:
train_data_scaled2=pd.DataFrame(train_ratio_standard_scaled_values2)
train_data_scaled2.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-0.002879,0.138669,-0.030606,-1.446295,-0.393016,4.007701,-3.018347,-0.045073,-0.200194,-0.272088,...,-2.304077,0.040981,-5.497208,-0.099644,4.256389,0.717136,-0.447807,1.123831,-0.541858,0.387296
1,-0.002820,-0.011573,-0.029945,-1.442120,-0.435582,3.740854,-2.867722,-0.039458,-0.193953,-0.263098,...,-2.411203,0.057977,-5.233175,0.263078,4.077955,0.884672,-0.347545,1.073763,-0.827164,0.476098
2,-0.002824,0.014184,-0.030395,-1.431949,-0.472478,3.470171,-2.714716,-0.037605,-0.188426,-0.256148,...,-2.383503,0.061263,-5.016467,0.354689,3.913188,0.909657,-0.296740,1.019853,-0.886323,0.493816
3,-0.002840,0.014184,-0.030468,-1.430999,-0.515224,3.189484,-2.558857,-0.035150,-0.183609,-0.251349,...,-2.296284,0.057437,-4.820204,0.300679,3.754800,0.859873,-0.320819,0.937773,-0.849010,0.499663
4,-0.002842,0.035219,-0.030888,-1.420989,-0.552272,2.918904,-2.405917,-0.033202,-0.178162,-0.244913,...,-2.233029,0.057074,-4.615365,0.325057,3.594228,0.849312,-0.282955,0.883555,-0.850530,0.501443


In [154]:
test_ratio_standard_scaled_values2 = standard_scaler.fit_transform(test_data_pca_df2.values)
test_data_scaled2=pd.DataFrame(test_ratio_standard_scaled_values2)
test_data_scaled2.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-0.012755,-0.037075,0.040349,-1.057800,-1.001226,1.110979,0.208551,0.018873,0.042004,-1.089026,...,-0.014346,-0.428840,0.208092,-0.412511,-0.244829,0.576702,-0.057732,-0.058105,0.166011,-0.188376
1,-0.012755,-0.037075,0.040349,-1.057800,-1.001226,1.110979,0.208551,0.018873,0.042004,-1.089026,...,-0.014346,-0.428840,0.208092,-0.412511,-0.244829,0.576702,-0.057732,-0.058105,0.166011,-0.188376
2,-0.012755,-0.037075,0.040349,-1.057800,-1.001226,1.110979,0.208551,0.018873,0.042004,-1.089026,...,-0.014346,-0.428840,0.208092,-0.412511,-0.244829,0.576702,-0.057732,-0.058105,0.166011,-0.188376
3,-0.012755,-0.037075,0.040011,-1.053497,-0.999374,1.108655,0.207935,0.018834,0.041820,-1.084111,...,-0.014566,-0.429114,0.208051,-0.412566,-0.244915,0.576985,-0.058124,-0.057988,0.166426,-0.187950
4,-0.012755,-0.037075,0.040011,-1.053497,-0.999374,1.108655,0.207935,0.018834,0.042032,-1.091198,...,-0.010124,-0.437004,0.220135,-0.347389,-0.239690,0.513516,-0.061896,-0.052876,0.197469,-0.180489


#Classification:

**1 Using SVM Algorithm**

In [0]:
clf = svm.SVC(kernel='linear',class_weight="balanced", max_iter=100000000)

In [162]:
clf.fit(train_data_scaled2, train_target_decoded_m[0])  # train the model by using training set with target labelled data

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=100000000, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [0]:
clf_predict = clf.predict(test_data_scaled2)  # prediciton the test dataset's output

In [165]:
len(clf_predict)

311029

In [164]:
print(clf.score(test_data_scaled2, test_target_decoded_m))

0.8778216822225581
